## Import

In [ ]:
# reload updated module
%load_ext autoreload
%autoreload 2

# make screen full width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from train import *
from pytorch_lightning.callbacks import EarlyStopping

## Model

In [5]:
model = BiLSTM_CRF_PL(
    n_vocab1 = 13907, n_vocab2 = 0, n_vocab3 = 0, n_label = 5,
    emb1_dim = 512, emb2_dim = 0, emb3_dim = 0, hid_dim = 1024,
    m_type = 'sy', data_name = 'sy_1', lr = 0.001, batch_size=100)

## Train

In [8]:
model.hparams.lr = 0.0005

In [ ]:
trainer = pl.Trainer(
    max_epochs=3, 
    log_every_n_steps=1,
    flush_logs_every_n_steps=1,
    gpus=1, 
    weights_summary=None, 
    progress_bar_refresh_rate=10)
trainer.fit(model)

In [ ]:
trainer = pl.Trainer(
    max_epochs=1, 
    limit_train_batches=50, 
    limit_val_batches=20,
    log_every_n_steps=1,
    flush_logs_every_n_steps=5,
    gpus=1, 
    weights_summary=None, 
    progress_bar_refresh_rate=5)
trainer.fit(model)

## Train from Checkpoint

In [ ]:
# resume from check point (must initiate the model first)
trainer = pl.Trainer(
    max_epochs=10, 
    limit_train_batches=10, 
    limit_val_batches=3,
    gpus=1, 
    weights_summary=None, 
    progress_bar_refresh_rate=5,
    resume_from_checkpoint='lightning_logs/version_109/checkpoints/epoch=9-step=99.ckpt')
trainer.fit(model)

## Load from Checkpoint

In [ ]:
model = BiLSTM_CRF_PL.load_from_checkpoint(
    'lightning_logs/version_31/checkpoints/epoch=0-step=19.ckpt')
print(model.hparams)

## Test

In [10]:
result = trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'char f1': 0.9795994153201912,
 'char precision': 0.9750617361624487,
 'char recall': 0.98417952625897,
 'word f1': 0.9485007703551535,
 'word precision': 0.9441071456658855,
 'word recall': 0.9529354797739252}
--------------------------------------------------------------------------------


## Tools

### Debugging

In [ ]:
# touch all the code to find bugs
trainer = pl.Trainer(
    max_epochs=20, 
    fast_dev_run=True, # here
    limit_train_batches=1, 
    limit_val_batches=1,
    gpus=1, 
    weights_summary=None)
trainer.fit(model)

In [ ]:
# train small number of batchs
trainer = pl.Trainer(
    max_epochs=20, 
    limit_train_batches=10, # here
    limit_val_batches=1, # here
    log_every_n_steps=1,
    flush_logs_every_n_steps=1,
    gpus=1, 
    weights_summary=None)
trainer.fit(model)

In [ ]:
# train only 10% of an epoch
trainer = pl.Trainer(
    max_epochs=3, 
    limit_train_batches=0.1, # here
    log_every_n_steps=1,
    flush_logs_every_n_steps=1,
    gpus=1, 
    weights_summary=None)
trainer.fit(model)

In [ ]:
# for large batch, run validation every 25% of a training epoch
trainer = pl.Trainer(
    max_epochs=3, 
    val_check_interval=0.25, # here
    limit_val_batches=1,
    log_every_n_steps=1,
    gpus=1, 
    weights_summary=None)
trainer.fit(model)

In [ ]:
# Profile your code to find speed/memory bottlenecks
pl.Trainer(profiler="simple")

### Find LR and BS

In [ ]:
# search lr
lr_finder = trainer.tuner.lr_find(model)
dfig = lr_finder.plot(suggest=True)

In [14]:
# set
model.hparams.lr = 0.0001

In [ ]:
# search batch_size
trainer = pl.Trainer(gpus=1)
tuner = trainer.tuner.scale_batch_size(model)

In [27]:
# set
model.hparams.batch_size = 512

### Callbacks: EarlyStopping

In [29]:
early_stop_callback = EarlyStopping(
   monitor='val_acc',
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode='max')

In [ ]:
trainer = pl.Trainer(
    max_epochs=30, 
    limit_train_batches=5, 
    limit_val_batches=3,
    log_every_n_steps=1,
    flush_logs_every_n_steps=1,
    callbacks=[early_stop_callback], # here
    gpus=1, 
    weights_summary=None, 
    progress_bar_refresh_rate=1)
trainer.fit(model)

### Callbacks: LearningRateMonitor

In [ ]:
# basic
def configure_optimizers(self):
   optimizer = Adam(...)
   scheduler = LambdaLR(optimizer, ...)
   return [optimizer], [scheduler]

# when scheduler require lr monitor, use callback
lr_monitor = LearningRateMonitor(logging_interval='step')
trainer = Trainer(callbacks=[lr_monitor])

# The ReduceLROnPlateau scheduler requires a monitor
def configure_optimizers(self):
   return {
       'optimizer': Adam(...),
       'lr_scheduler': ReduceLROnPlateau(optimizer, ...),
       'monitor': 'metric_to_track'
   }

## Technique

### Freeze layers

In [65]:
model = BiLSTM_CRF_PL(n_vocab1 = 13907)

In [ ]:
# check weight name of the model
for weight_name in model.state_dict():
    print(weight_name)

In [67]:
# freeze the embedding weight
model.x1emb.weight.requires_grad = False

In [ ]:
# observe embeding weight before training
model.x1emb.weight[0][:5]

In [ ]:
# observe weight of other layer (to be changed)
model.lstm.weight_ih_l0[0][:5]

In [ ]:
# weight summary shows Non-trainable params
trainer = pl.Trainer(
    max_epochs=1, 
    limit_train_batches=10, 
    limit_val_batches=1,
    gpus=1)
trainer.fit(model)

In [ ]:
# weight is unchanged after trainning
model.x1emb.weight[0][:5]

In [ ]:
# weight of other layer changes
model.lstm.weight_ih_l0[0][:5]

### Import weight from pretrained

In [77]:
# load pretrained model from checkpoint
pretrained_model = BiLSTM_CRF_PL.load_from_checkpoint(
    'lightning_logs/version_45/checkpoints/epoch=0-step=9.ckpt')

In [ ]:
# look at pretrained embedding weight
pretrained_model.x1emb.weight[0][:5]

In [87]:
# define new model
new_model = BiLSTM_CRF_PL(n_vocab1 = 13907)

In [ ]:
# explore embedding weight (different)
new_model.x1emb.weight[0][:5]

In [89]:
# set the weight of new_model = the weight of pretrained_model
new_model.x1emb.weight.data.copy_ = pretrained_model.x1emb.weight

In [ ]:
# embedding weight has changed
new_model.x1emb.weight[0][:5]

In [ ]:
# train
trainer = pl.Trainer(
    max_epochs=1, 
    limit_train_batches=10, 
    limit_val_batches=1,
    gpus=1)
trainer.fit(new_model)

In [93]:
# embedding weight has changed
new_model.x1emb.weight[0][:5]

tensor([-0.2877, -0.7774, -1.9446,  0.8272,  0.4591], grad_fn=<SliceBackward>)

get glove embedding<br> 
https://stackoverflow.com/questions/37793118/load-pretrained-glove-vectors-in-python